In [1]:

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import math
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.preparation import fetch_dataset
from src.processing import split_train_test_val, dataset_to_array, dataset_to_generator, make_tf_dataset
from src.modules.visualizer import do_heatmap
import numpy as np
from src.processing import make_tf_dataset, split_train_test


In [2]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import LSTM
from tensorflow.compat.v1.math import confusion_matrix


In [3]:
# This is the size of the window that is fed into the DNN
window_size = 15
# The number of the features present in the dataset
num_of_features = 42
# Number of distinct labels in the output
label_length = 4
# Hyperparameter that defines the number of samples to work through
# before updating the internal model parameters.
batch_size = 25
# Epochs
ep = 50

In [4]:
# Create a model
model = Sequential()
model.add(LSTM(512, return_sequences=False, input_shape=(window_size, num_of_features)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(label_length, activation='softmax'))
model.compile(loss='binary_crossentropy',
                   optimizer='adam',
                   metrics=['accuracy',])


In [5]:

def list_to_num(el):
    if el == [1,0,0,0]:
        return 0
    elif el == [0,1,0,0]:
        return 1
    elif el == [0,0,1,0]:
        return 2
    elif el == [0,0,0,1]:
        return 3
    else:
        print('Error',el)
        
def test_model(md, dt):
    print('\n')
    data_test = make_tf_dataset(dt, window_size, num_of_features, label_length ).batch(batch_size)
    results = md.evaluate(data_test)
    
    ph = []
    predictions = md.predict(data_test)
    for pred in predictions:
        i = list(pred).index(max(list(pred)))
        ph.append(i)
    # Ground truth
    fm , labels = dataset_to_array(dt)
    # Padding
    ph = ph + [3 for i in range (0, len(labels) - len(ph))]
    
    # -----------------
    labels = [list_to_num(list(x)) for x in labels]
    # CM
    cm = confusion_matrix(labels, ph)
    print('\n',cm)
    
def build_and_test(dataset, name):
    train, test, val = split_train_test_val(dataset)

    data_train = make_tf_dataset(train, window_size, num_of_features, label_length ).batch(batch_size)
    data_test = make_tf_dataset(test ,window_size, num_of_features, label_length).batch(batch_size)

    model.fit(data_train, epochs=ep)
    data_val = make_tf_dataset(val ,window_size, num_of_features, label_length).batch(batch_size)

    results = model.evaluate(data_val)
    test_model(model, val)
    model.save('model_{0}_{1}.h5'.format(name, ep))  # creates a HDF5 file 'my_model.h5'

    
    


In [6]:
dataset = fetch_dataset('dataset_a1_slim.pkl')

build_and_test(dataset, 'a1_slim')



W0802 20:19:41.818298 4690019776 deprecation.py:323] From /Users/noresources/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:505: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
W0802 20:19:41.976528 4690019776 deprecation.py:323] From /Use

Epoch 1/50
241/241 [==============================] - 51s 212ms/step - loss: 0.1451 - accuracy: 0.9529
Epoch 2/50
241/241 [==============================] - 66s 276ms/step - loss: 0.0671 - accuracy: 0.9796
Epoch 3/50
241/241 [==============================] - 84s 349ms/step - loss: 0.0410 - accuracy: 0.9889
Epoch 4/50
241/241 [==============================] - 65s 269ms/step - loss: 0.0315 - accuracy: 0.9904
Epoch 5/50
241/241 [==============================] - 73s 302ms/step - loss: 0.0276 - accuracy: 0.9917
Epoch 6/50
241/241 [==============================] - 58s 242ms/step - loss: 0.0202 - accuracy: 0.9946
Epoch 7/50
241/241 [==============================] - 56s 233ms/step - loss: 0.0132 - accuracy: 0.9967
Epoch 8/50
241/241 [==============================] - 84s 348ms/step - loss: 0.0112 - accuracy: 0.9970
Epoch 9/50
241/241 [==============================] - 59s 247ms/step - loss: 0.0087 - accuracy: 0.9977
Epoch 10/50
241/241 [==============================] - 49s 202ms/step - l

In [7]:
dataset = fetch_dataset('dataset_f1_slim.pkl')

build_and_test(dataset, 'f1_slim')



Epoch 1/50
1621/1621 [==============================] - 359s 221ms/step - loss: 0.2694 - accuracy: 0.9052
Epoch 2/50
1621/1621 [==============================] - 386s 238ms/step - loss: 0.1682 - accuracy: 0.9357
Epoch 3/50
1621/1621 [==============================] - 293s 181ms/step - loss: 0.1482 - accuracy: 0.9430
Epoch 4/50
1621/1621 [==============================] - 288s 178ms/step - loss: 0.1323 - accuracy: 0.9486
Epoch 5/50
1621/1621 [==============================] - 288s 178ms/step - loss: 0.1210 - accuracy: 0.9531
Epoch 6/50
1621/1621 [==============================] - 286s 176ms/step - loss: 0.1105 - accuracy: 0.9566
Epoch 7/50
1621/1621 [==============================] - 286s 176ms/step - loss: 0.1033 - accuracy: 0.9607
Epoch 8/50
1621/1621 [==============================] - 286s 176ms/step - loss: 0.0959 - accuracy: 0.9635
Epoch 9/50
1621/1621 [==============================] - 305s 188ms/step - loss: 0.0868 - accuracy: 0.9673
Epoch 10/50
1621/1621 [=======================

In [8]:
dataset = fetch_dataset('dataset_g1_slim.pkl')

build_and_test(dataset, 'g1_slim')



Epoch 1/50
509/509 [==============================] - 122s 241ms/step - loss: 0.1560 - accuracy: 0.9647
Epoch 2/50
509/509 [==============================] - 109s 215ms/step - loss: 0.0389 - accuracy: 0.9886
Epoch 3/50
509/509 [==============================] - 103s 202ms/step - loss: 0.0220 - accuracy: 0.9933
Epoch 4/50
509/509 [==============================] - 98s 192ms/step - loss: 0.0136 - accuracy: 0.9959
Epoch 5/50
509/509 [==============================] - 95s 188ms/step - loss: 0.0096 - accuracy: 0.9968
Epoch 6/50
509/509 [==============================] - 97s 191ms/step - loss: 0.0083 - accuracy: 0.9972
Epoch 7/50
509/509 [==============================] - 97s 191ms/step - loss: 0.0062 - accuracy: 0.9981
Epoch 8/50
509/509 [==============================] - 128s 252ms/step - loss: 0.0053 - accuracy: 0.9984
Epoch 9/50
509/509 [==============================] - 105s 207ms/step - loss: 0.0028 - accuracy: 0.9991
Epoch 10/50
509/509 [==============================] - 96s 189ms/ste